In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import json
import gensim.downloader as api
from sklearn.metrics import f1_score
import numpy as np
from gensim.models import KeyedVectors
import pickle
import matplotlib.pyplot as plt

In [2]:
with open('Json Task1/NER_train.json', 'r') as f:
    task1_train_data = json.load(f)

with open('Json Task1/NER_val.json', 'r') as f:
    task1_val_data = json.load(f)

with open('Json Task1/NER_test.json', 'r') as f:
    task1_test_data = json.load(f)

with open('Json Task2/ATE_train.json', 'r') as f:
    task2_train_data = json.load(f)

with open('Json Task2/ATE_val.json', 'r') as f:
    task2_val_data = json.load(f)

with open('Json Task2/ATE_test.json', 'r') as f:
    task2_test_data = json.load(f)

with open('glove_embedding.pkl', 'rb') as pickle_file:
    glove_embeddings = pickle.load(pickle_file)
    
with open('fast_text_embedding.pkl', 'rb') as pickle_file:
    fast_text_embedding = pickle.load(pickle_file)

bio_mapping_task1 = {'B_ORG': 0, 'I_ORG': 1, 'B_RESPONDENT': 2, 'I_RESPONDENT': 3, 'B_JUDGE': 4, 'I_JUDGE': 5,
               'B_STATUTE': 6, 'I_STATUTE': 7, 'B_OTHER_PERSON': 8, 'I_OTHER_PERSON': 9, 'B_COURT': 10, 'I_COURT': 11,
               'B_GPE': 12, 'I_GPE': 13, 'B_PETITIONER': 14, 'I_PETITIONER': 15, 'B_WITNESS': 16, 'I_WITNESS': 17,
               'B_CASE_NUMBER': 18, 'I_CASE_NUMBER': 19, 'B_PRECEDENT': 20, 'I_PRECEDENT': 21, 'B_DATE': 22, 'I_DATE': 23,
               'B_PROVISION': 24, 'I_PROVISION': 25, 'O': 26}
bio_mapping_task2 = {'O' : 0, 'I' : 1, 'B' : 2}

In [3]:
word2vec_model = api.load('word2vec-google-news-300')

word2vec = torch.FloatTensor(word2vec_model.vectors)
word2vec_u_ = np.concatenate((word2vec, np.zeros((1, 300), dtype = 'float32')), axis = 0)

# Add an unknown token to the vocabulary
word_to_index = {word: index for index, word in enumerate(word2vec_model.index_to_key)}
word_to_index['<unk>'] = len(word_to_index)

# Example usage
unknown_token_index = word_to_index['<unk>']

In [2]:
# fasttext_model_path = "wiki-news-300d-1M-subword.vec"
# fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path, binary=False)

# def get_word_embedding(word):
#     if word in fasttext_model:
#         return fasttext_model[word]
#     else:
#         subword_embeddings = [fasttext_model[subword] for subword in fasttext_model.key_to_index if subword in word]
#         if subword_embeddings:
#             return np.mean(subword_embeddings, axis=0)
#         else:
#             return np.zeros(fasttext_model.vector_size)

# fast_text_embedding = {}
# def get_word_vector():
#     count = 0
#     data = [task1_train_data, task1_val_data, task1_test_data, task2_train_data, task2_val_data, task2_test_data]
#     for task_data in data:
#         for key in task_data:
#             words = task_data[key]['text'].split(' ')
#             print(count)
#             count +=1
#             for word in words:
#                 if word not in fast_text_embedding:
#                     word_vector = get_word_embedding(word)
#                     fast_text_embedding[word] = word_vector


# get_word_vector()  

# def load_glove_embeddings(file_path):
#     word_embeddings = {}
#     with open(file_path, 'r', encoding='utf-8') as file:
#         for line in file:
#             try:
#                 values = line.split()
#                 word = values[0]
#                 vector = np.array(values[1:], dtype='float32')
#                 word_embeddings[word] = vector
#             except ValueError as e:
#                 print(f"Error processing line: {line}\nError: {e}")
#     return word_embeddings

# glove_file_path = 'glove.840B.300d.txt'  # Adjust the path based on your downloaded file
# glove_embeddings = load_glove_embeddings(glove_file_path)

# g_e = {}
# def get_glove_dictionary(glove_embeddings):
#     count = 0
#     data = [task1_train_data, task1_val_data, task1_test_data, task2_train_data, task2_val_data, task2_test_data]
#     for task_data in data:
#         for key in task_data:
#             words = task_data[key]['text'].split(' ')
#             print(count)
#             count +=1
#             for word in words:
#                 if word not in g_e:
#                     g_e[word] = glove_embeddings.get(word, np.zeros(300, dtype = 'float32'))



# get_glove_dictionary(glove_embeddings)

In [12]:
# import pickle

# with open('glove_embedding.json', 'w') as json_file:
#     json.dump(g_e, json_file)

# with open('fast_text_embedding.json', 'w') as json_file:
#     json.dump(fast_text_embedding, json_file)

# with open('glove_embedding.pkl', 'wb') as pickle_file:
#     pickle.dump(g_e, pickle_file)

# with open('fast_text_embedding.pkl', 'wb') as pickle_file:
#     pickle.dump(fast_text_embedding, pickle_file)


In [4]:

class GRUModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(GRUModel, self).__init__()
        # self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)
        self.gru = nn.GRU(embedding_dim, 128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.gru(x)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

class Task_data(Dataset):
    def __init__(self, data, bio_index, embedding_type):
        self.data = data
        self.length = len(self.data)
        self.bio_index =  bio_index
        self.embedding_type = embedding_type

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        input_sequence = self.data[str(index)]['text'].split(' ')
        sentence_embeddings = []
        if self.embedding_type == "glove":
            sentence_embeddings = [glove_embeddings.get(word, np.zeros(300, dtype = 'float32')) for word in input_sequence]
        elif self.embedding_type == "word2vec":
            sentence_embeddings = [word2vec_u_[word_to_index.get(word, word_to_index['<unk>'])] for word in input_sequence]
        elif self.embedding_type == "fast_text":
            sentence_embeddings = [np.array(fast_text_embedding.get(word, np.zeros(300, dtype = 'float32')), dtype = 'float32') for word in input_sequence]

        sentence_embeddings = np.array(sentence_embeddings, dtype='float32')
        output_sequence = self.data[str(index)]['labels']
        output_labels = [self.bio_index[word] for word in output_sequence]

        # output_labels = np.array(output_labels, dtype='float32')
        return torch.tensor(sentence_embeddings), torch.tensor(output_labels)

In [15]:
def train_model(task, embedding_type, model, optimizer, criterion, device,  num_epochs = 15, batch_size = 1):
    train_dataloader  = None
    val_dataloader  = None
    test_dataloader = None

    if task == 1:
        train_dataloader = DataLoader(Task_data(task1_train_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=True)
        val_dataloader =  DataLoader(Task_data(task1_val_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=True)
    elif task == 2:
        train_dataloader = DataLoader(Task_data(task2_train_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=True)
        val_dataloader =  DataLoader(Task_data(task2_val_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=True)


    train_losses = []
    val_losses = []
    train_f1_scores = []
    val_f1_scores = []

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        total_train_loss = 0
        all_train_predictions = []
        all_train_targets = []

        for batch_idx, (inputs, targets) in enumerate(train_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            
            loss = 0
            for i in range(outputs.size(1)):  # Iterate over time steps
                loss += criterion(outputs[:, i, :], targets[:, i])  # Apply CrossEntropyLoss at each time step
            
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

            all_train_predictions.extend(outputs.argmax(dim=2).view(-1).cpu().numpy())
            all_train_targets.extend(targets.view(-1).cpu().numpy())


        avg_train_loss = total_train_loss / len(train_dataloader)
        train_losses.append(avg_train_loss)

        train_macro_f1 = f1_score(all_train_targets, all_train_predictions, average='macro')
        train_f1_scores.append(train_macro_f1)
        print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Training Macro F1-Score: {train_macro_f1}")

        model.eval()  # Set the model to evaluation mode
        total_val_loss = 0
        all_val_predictions = []
        all_val_targets = []

        with torch.no_grad():
            for val_inputs, val_targets in val_dataloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = model(val_inputs)

                loss = 0
                for i in range(val_outputs.size(1)):  # Iterate over time steps
                    loss += criterion(val_outputs[:, i, :], val_targets[:, i])  

                total_val_loss += loss.item()

                all_val_predictions.extend(val_outputs.argmax(dim=2).view(-1).cpu().numpy())
                all_val_targets.extend(val_targets.view(-1).cpu().numpy())

            avg_val_loss = total_val_loss / len(val_dataloader)
            val_losses.append(avg_val_loss)

            val_macro_f1 = f1_score(all_val_targets, all_val_predictions, average='macro')
            val_f1_scores.append(val_macro_f1)
        print(f"Epoch {epoch + 1},  Validation Loss: {avg_val_loss}, Validation Macro F1-Score: {val_macro_f1}")

    return train_losses, train_f1_scores, val_losses, val_f1_scores

def test_model(task, embedding_type, model, criterion, device, batch_size = 1):
    test_dataloader = None
    if task == 1:
        test_dataloader =  DataLoader(Task_data(task1_test_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=False)

    elif task == 2:
        test_dataloader =  DataLoader(Task_data(task2_test_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=False)

    total_test_loss = 0
    all_test_predictions = []
    all_test_targets = []

    with torch.no_grad():
        for test_inputs, test_targets in test_dataloader:
            test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            test_outputs = model(test_inputs)

            loss = 0
            for i in range(test_outputs.size(1)):  # Iterate over time steps
                loss += criterion(test_outputs[:, i, :], test_targets[:, i])  

            total_test_loss += loss.item()

            all_test_predictions.extend(test_outputs.argmax(dim=2).view(-1).cpu().numpy())
            all_test_targets.extend(test_targets.view(-1).cpu().numpy())

        avg_test_loss = total_test_loss / len(test_dataloader)
        test_macro_f1 = f1_score(all_test_targets, all_test_predictions, average='macro')
    print(f'Test Loss: {avg_test_loss}, Test Macro F1-Score: {test_macro_f1}')

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

train_losses, train_f1_scores, val_losses, val_f1_scores = train_model(task = 1, embedding_type = "fast_text", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 1, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

torch.save(model, 'trained_models/task1_gru_fast_text.pt')


# plt.plot(range(1, 21), train_losses, label='Training Loss')
# plt.plot(range(1, 21), val_losses, label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# plt.plot(range(1, 21), train_f1_scores, label='Training F1')
# plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
# plt.xlabel('Epochs')
# plt.ylabel('F1-score')
# plt.legend()
# plt.show()

Epoch 1, Training Loss: 19.642157192567204, Training Macro F1-Score: 0.06781638314613055
Epoch 1,  Validation Loss: 15.606822148470373, Validation Macro F1-Score: 0.11126282637394767
Epoch 2, Training Loss: 13.566507858955736, Training Macro F1-Score: 0.19538653408764303
Epoch 2,  Validation Loss: 12.359317899785033, Validation Macro F1-Score: 0.2617143961875173
Epoch 3, Training Loss: 11.412781864595049, Training Macro F1-Score: 0.2890044971297508
Epoch 3,  Validation Loss: 11.447341237489804, Validation Macro F1-Score: 0.31511825171651175
Epoch 4, Training Loss: 10.227397550206083, Training Macro F1-Score: 0.3481774093864619
Epoch 4,  Validation Loss: 10.234689493350034, Validation Macro F1-Score: 0.36609368324819985
Epoch 5, Training Loss: 9.45445255273151, Training Macro F1-Score: 0.38429615199547373
Epoch 5,  Validation Loss: 9.29385928013822, Validation Macro F1-Score: 0.39257778708318297
Epoch 6, Training Loss: 8.876228244815751, Training Macro F1-Score: 0.40435280871700674
Epoc

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores =  train_model(task = 1, embedding_type = "glove", model = model, optimizer=optimizer, num_epochs=9,criterion=criterion, device=device)
test_model(task = 1, embedding_type = "glove", model = model, criterion=criterion, device=device)

torch.save(model, 'trained_models/task1_gru_glove.pt')


# plt.plot(range(1, 10), train_losses, label='Training Loss')
# plt.plot(range(1, 10), val_losses, label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# plt.plot(range(1, 10), train_f1_scores, label='Training F1')
# plt.plot(range(1, 10), val_f1_scores, label='Validation F1')
# plt.xlabel('Epochs')
# plt.ylabel('F1-score')
# plt.legend()
# plt.show()

Epoch 1, Training Loss: 14.22793648684082, Training Macro F1-Score: 0.24863933258760887
Epoch 1,  Validation Loss: 10.305158162481812, Validation Macro F1-Score: 0.35087893898350714
Epoch 2, Training Loss: 9.032673935542363, Training Macro F1-Score: 0.3821619715854325
Epoch 2,  Validation Loss: 8.73616161979933, Validation Macro F1-Score: 0.3921624102743777
Epoch 3, Training Loss: 7.842695083700581, Training Macro F1-Score: 0.429368257013146
Epoch 3,  Validation Loss: 8.161109808593785, Validation Macro F1-Score: 0.43239811803304995
Epoch 4, Training Loss: 7.099596732354082, Training Macro F1-Score: 0.48501658857872065
Epoch 4,  Validation Loss: 7.758175951882245, Validation Macro F1-Score: 0.4855453466289377
Epoch 5, Training Loss: 6.512021835892337, Training Macro F1-Score: 0.535269106156463
Epoch 5,  Validation Loss: 7.749140801371142, Validation Macro F1-Score: 0.5102283690160417
Epoch 6, Training Loss: 6.047801629319921, Training Macro F1-Score: 0.5794892695777419
Epoch 6,  Valida

In [18]:
model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores =  train_model(task = 1, embedding_type = "word2vec", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 1, embedding_type = "word2vec", model = model, criterion=criterion, device=device)

torch.save(model, 'trained_models/task1_gru_word2vec.pt')


# plt.plot(range(1, 21), train_losses, label='Training Loss')
# plt.plot(range(1, 21), val_losses, label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# plt.plot(range(1, 21), train_f1_scores, label='Training F1')
# plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
# plt.xlabel('Epochs')
# plt.ylabel('F1-score')
# plt.legend()
# plt.show()

Epoch 1, Training Loss: 17.270272484191114, Training Macro F1-Score: 0.1430320728110131
Epoch 1,  Validation Loss: 13.147391776471697, Validation Macro F1-Score: 0.23977491628174316
Epoch 2, Training Loss: 11.601458160648168, Training Macro F1-Score: 0.2935116946700014
Epoch 2,  Validation Loss: 11.039870294633845, Validation Macro F1-Score: 0.3175315638402801
Epoch 3, Training Loss: 10.215917539520824, Training Macro F1-Score: 0.34805811283289745
Epoch 3,  Validation Loss: 10.23861379537056, Validation Macro F1-Score: 0.35918442442904763
Epoch 4, Training Loss: 9.465163666504466, Training Macro F1-Score: 0.3878833884401204
Epoch 4,  Validation Loss: 9.76614810425668, Validation Macro F1-Score: 0.40348721402882504
Epoch 5, Training Loss: 8.902044474147349, Training Macro F1-Score: 0.4265061925510285
Epoch 5,  Validation Loss: 9.503681607867984, Validation Macro F1-Score: 0.43205636967927313
Epoch 6, Training Loss: 8.477939453421822, Training Macro F1-Score: 0.45737032967614205
Epoch 6,

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores = train_model(task = 2, embedding_type = "fast_text", model = model, optimizer=optimizer, criterion=criterion, device=device)

test_model(task = 2, embedding_type = "fast_text", model = model, criterion=criterion, device=device)
torch.save(model, 'trained_models/task2_gru_fast_text.pt')

# plt.plot(range(1, 21), train_losses, label='Training Loss')
# plt.plot(range(1, 21), val_losses, label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# plt.plot(range(1, 21), train_f1_scores, label='Training F1')
# plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
# plt.xlabel('Epochs')
# plt.ylabel('F1-score')
# plt.legend()
# plt.show()

Epoch 1, Training Loss: 13.87219355832662, Training Macro F1-Score: 0.2316952097294614
Epoch 1,  Validation Loss: 7.40538707062534, Validation Macro F1-Score: 0.31532441302701336
Epoch 2, Training Loss: 6.387104538102813, Training Macro F1-Score: 0.32548232858679443
Epoch 2,  Validation Loss: 4.897889967378416, Validation Macro F1-Score: 0.38712255613497487
Epoch 3, Training Loss: 4.706549936769814, Training Macro F1-Score: 0.4005375712650656
Epoch 3,  Validation Loss: 3.847001170458859, Validation Macro F1-Score: 0.6987163043570762
Epoch 4, Training Loss: 4.0780225039745535, Training Macro F1-Score: 0.4928218774378933
Epoch 4,  Validation Loss: 3.525607310881898, Validation Macro F1-Score: 0.6890542918922566
Epoch 5, Training Loss: 3.839100873027877, Training Macro F1-Score: 0.5169626984733329
Epoch 5,  Validation Loss: 3.443792109620081, Validation Macro F1-Score: 0.6955290435977818
Epoch 6, Training Loss: 3.6714536016334103, Training Macro F1-Score: 0.5287674597354581
Epoch 6,  Vali

In [20]:
model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores = train_model(task = 2, embedding_type = "glove", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 2, embedding_type = "glove", model = model, criterion=criterion, device=device)
torch.save(model, 'trained_models/task2_gru_glove.pt')


# plt.plot(range(1, 21), train_losses, label='Training Loss')
# plt.plot(range(1, 21), val_losses, label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# plt.plot(range(1, 21), train_f1_scores, label='Training F1')
# plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
# plt.xlabel('Epochs')
# plt.ylabel('F1-score')
# plt.legend()
# plt.show()

Epoch 1, Training Loss: 11.825610663190846, Training Macro F1-Score: 0.09803017547457402
Epoch 1,  Validation Loss: 5.070290366536406, Validation Macro F1-Score: 0.4311739782852617
Epoch 2, Training Loss: 4.395027361251002, Training Macro F1-Score: 0.466800553315176
Epoch 2,  Validation Loss: 3.448419916915567, Validation Macro F1-Score: 0.7555243791835488
Epoch 3, Training Loss: 3.442161086691794, Training Macro F1-Score: 0.3753955734864845
Epoch 3,  Validation Loss: 3.0352668461802343, Validation Macro F1-Score: 0.7901949549195312
Epoch 4, Training Loss: 3.0489664346341527, Training Macro F1-Score: 0.46731920581466324
Epoch 4,  Validation Loss: 2.9745626295470213, Validation Macro F1-Score: 0.8014423259660003
Epoch 5, Training Loss: 2.732748146672583, Training Macro F1-Score: 0.6090813583343915
Epoch 5,  Validation Loss: 2.7967684196486866, Validation Macro F1-Score: 0.7869728017194726
Epoch 6, Training Loss: 2.508787012809094, Training Macro F1-Score: 0.6173067233408315
Epoch 6,  Va

In [21]:
model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores = train_model(task = 2, embedding_type = "word2vec", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 2, embedding_type = "word2vec", model = model, criterion=criterion, device=device)
torch.save(model, 'trained_models/task2_gru_word2vec.pt')

# plt.plot(range(1, 21), train_losses, label='Training Loss')
# plt.plot(range(1, 21), val_losses, label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# plt.plot(range(1, 21), train_f1_scores, label='Training F1')
# plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
# plt.xlabel('Epochs')
# plt.ylabel('F1-score')
# plt.legend()
# plt.show()

Epoch 1, Training Loss: 13.356570194362279, Training Macro F1-Score: 0.1855535381790909
Epoch 1,  Validation Loss: 6.713199974739388, Validation Macro F1-Score: 0.31532441302701336
Epoch 2, Training Loss: 5.442057884423675, Training Macro F1-Score: 0.47309608818015425
Epoch 2,  Validation Loss: 3.982285129561272, Validation Macro F1-Score: 0.6260359561651426
Epoch 3, Training Loss: 4.013988341338456, Training Macro F1-Score: 0.6894811739582409
Epoch 3,  Validation Loss: 3.36503385027794, Validation Macro F1-Score: 0.704163114409497
Epoch 4, Training Loss: 3.470639836659889, Training Macro F1-Score: 0.7382322464847361
Epoch 4,  Validation Loss: 3.1252640600754247, Validation Macro F1-Score: 0.7632677473683224
Epoch 5, Training Loss: 3.24325854121142, Training Macro F1-Score: 0.7662988179605127
Epoch 5,  Validation Loss: 2.916542452083875, Validation Macro F1-Score: 0.7876794791599261
Epoch 6, Training Loss: 3.0697696701465196, Training Macro F1-Score: 0.7776899639813504
Epoch 6,  Valida